In [43]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from datetime import datetime, timedelta
import time, re, json, random

sns.set(
    font="Noto Sans",
    font_scale=1.5,
    style="whitegrid"
)
sns.set_palette([
    "#083A42",
    "#28a399",
    "#801650",
    "#FF5E00"
])

standingOffers = pd.read_csv(
    "./data/dmo.csv",
    thousands=",",
    converters={"dmo_price": lambda x: float(re.sub(r"[$,]", "", x))}
)

dailyUsage = pd.DataFrame(
    columns=["distributor", "time_start", "time_end", "usage"],
    index=[i for i in range(48 * 5)]
)
dailyUsage.iloc[[i for i in range(48)], 0] = "Ausgrid"
dailyUsage.iloc[[i for i in range(48, 48 * 2)], 0] = "Endeavour"
dailyUsage.iloc[[i for i in range(48 * 2, 48 * 3)], 0] = "Energex"
dailyUsage.iloc[[i for i in range(48 * 3, 48 * 4)], 0] = "Essential Energy"
dailyUsage.iloc[[i for i in range(48 * 4, 48 * 5)], 0] = "SA Power Networks"
dailyUsage.time_start = [time.time() for time in pd.date_range("2024-01-01", periods=48, freq='30T')] * 5
dailyUsage.time_end = [time.time() for time in pd.date_range("2024-01-01 00:30:00", periods=48, freq='30T')] * 5
dailyUsage.usage = [
    0.2326, 0.2247, 0.2140, 0.1900, 0.1725, 0.1563, 0.1450, 0.1379, 0.1348, 0.1348, 0.1399, 0.1484, 0.1651, 0.1869, 0.2012, 0.2168, 0.2210, 0.2213, 0.2207, 0.2186, 0.2162, 0.2142, 0.2124, 0.2125, 0.2138, 0.2140, 0.2128, 0.2114, 0.2106, 0.2111, 0.2142, 0.2224, 0.2346, 0.2514, 0.2731, 0.2997, 0.3148, 0.3180, 0.3120, 0.3040, 0.2974, 0.2898, 0.2776, 0.2663, 0.2595, 0.2541, 0.2465, 0.2379,
    0.2959, 0.2865, 0.2601, 0.2252, 0.1991, 0.1805, 0.1692, 0.1628, 0.1619, 0.1647, 0.1740, 0.1864, 0.2078, 0.2336, 0.2473, 0.2642, 0.2682, 0.2651, 0.2653, 0.2642, 0.2621, 0.2604, 0.2589, 0.2607, 0.2623, 0.2626, 0.2617, 0.2636, 0.2677, 0.2728, 0.2826, 0.2999, 0.3201, 0.3445, 0.3679, 0.3984, 0.4127, 0.4124, 0.4021, 0.3912, 0.3791, 0.3656, 0.3483, 0.3255, 0.3194, 0.3196, 0.3144, 0.3062,
    0.1963, 0.1803, 0.1697, 0.1599, 0.1527, 0.1479, 0.1452, 0.1441, 0.1463, 0.1517, 0.1635, 0.1790, 0.2029, 0.2316, 0.2577, 0.2681, 0.2705, 0.2689, 0.2693, 0.2687, 0.2669, 0.2661, 0.2653, 0.2676, 0.2699, 0.2718, 0.2745, 0.2761, 0.2800, 0.2819, 0.2881, 0.2987, 0.3116, 0.3307, 0.3509, 0.3753, 0.3922, 0.3998, 0.3886, 0.3777, 0.3709, 0.3495, 0.3265, 0.3088, 0.2939, 0.2758, 0.2488, 0.2201,
    0.2723, 0.2719, 0.2638, 0.2516, 0.2309, 0.2069, 0.1882, 0.1764, 0.1714, 0.1712, 0.1793, 0.1924, 0.2147, 0.2365, 0.2458, 0.2582, 0.2597, 0.2536, 0.2535, 0.2512, 0.2486, 0.2468, 0.2439, 0.2411, 0.2429, 0.2430, 0.2416, 0.2388, 0.2387, 0.2401, 0.2452, 0.2557, 0.2685, 0.2854, 0.3138, 0.3493, 0.3708, 0.3723, 0.3605, 0.3474, 0.3365, 0.3233, 0.3147, 0.3091, 0.3114, 0.2981, 0.2856, 0.2802,
    0.2577, 0.3017, 0.2996, 0.2542, 0.2145, 0.1884, 0.1731, 0.1557, 0.1441, 0.1394, 0.1417, 0.1498, 0.1674, 0.1802, 0.2012, 0.2064, 0.2008, 0.1963, 0.1958, 0.2001, 0.2048, 0.2125, 0.2150, 0.2157, 0.2705, 0.3168, 0.3146, 0.2669, 0.2252, 0.1978, 0.1818, 0.1635, 0.1513, 0.1464, 0.1488, 0.1573, 0.1758, 0.1892, 0.2113, 0.2167, 0.2108, 0.2061, 0.2056, 0.2101, 0.2151, 0.2231, 0.2258, 0.2265
]
dailyUsage.to_csv("./data/daily_usage_without_controlled_load.csv", index=False)

with open("./data/relevant_plan_details.json", "r") as file:
    plans = json.load(file)

# convert to dataframe
plan_data = pd.DataFrame()
plan_data.index.name = "planId"

# price estimators
# def estimate_supply_charges(plan)

# iterate through plans
for plan in plans:
    contractDetails = plan.pop("contractDetails")
    pricingModel = contractDetails.pop("pricingModel")    
    
    # if pricingModel == "SINGLE_RATE":
    planId = plan.pop("planId")
    plan_data.loc[planId, plan.keys()] = plan.values()
    plan_data.at[planId, "pricingModel"] = pricingModel
    plan_data.at[planId, "hasControlledLoad"] = "controlledLoad" in contractDetails
    plan_data.at[planId, "hasDemandCharges"] = pricingModel
    plan_data.at[planId, "hasMultipleSingleRates"] = pricingModel
    plan_data.at[planId, "hasDiscounts"] = "discounts" in contractDetails
    plan_data.at[planId, "hasIncentives"] = "incentives" in contractDetails
    plan_data.at[planId, "hasEligibilityRequirements"] = "eligibility" in contractDetails
    plan_data.at[planId, "hasGreenPower"] = "greenPowerCharges" in contractDetails
    plan_data.at[planId, "hasSolar"] = "solarFeedInTariff" in contractDetails
    plan_data.at[planId, "tariffRulesCount"] = len(contractDetails["tariffPeriod"])

    # DMO benchmarks
    plan_data.at[planId, "dmoPrice"] = standingOffers[
        (standingOffers["distribution_zone"] == plan["distributor"]) &
        (standingOffers["controlled_load"] == plan_data.at[planId, "hasControlledLoad"])
    ].iloc[0]["dmo_price"]
    plan_data.at[planId, "annualUsage"] = standingOffers[
        (standingOffers["distribution_zone"] == plan["distributor"]) &
        (standingOffers["controlled_load"] == plan_data.at[planId, "hasControlledLoad"])
    ].iloc[0]["dmo_kw"]
    plan_data.at[planId, "annualCLUsage"] = standingOffers[
        (standingOffers["distribution_zone"] == plan["distributor"]) &
        (standingOffers["controlled_load"] == plan_data.at[planId, "hasControlledLoad"])
    ].iloc[0]["cl_kw"]
    
    # effectiveFrom = datetime.strptime(plan["effectiveFrom"], "%Y-%m-%dT%H:%M:%S.%fZ")
    # startYear = effectiveFrom.year    
    # total_coverage = 0
    
    # for period in contractDetails["tariffPeriod"]:
        # startMonth, startDay = period["startDate"].split("-")
        # startDate = datetime.strptime(f"{startYear}-{startMonth}-{startDay}", "%Y-%m-%d")
        # endMonth, endDay = period["endDate"].split("-")
        # endDate = datetime.strptime(f"{startYear}-{endMonth}-{endDay}", "%Y-%m-%d")        
        # if endDate <= startDate:
        #     endDate = endDate + timedelta(days=366)
        # days_covered = (endDate - startDate).days + 1
        # total_coverage = total_coverage + days_covered
            
plan_data



,brand,brandName,fuelType,type,customerType,displayName,effectiveFrom,distributor,pricingModel,hasControlledLoad,...,hasMultipleSingleRates,hasDiscounts,hasIncentives,hasEligibilityRequirements,hasGreenPower,hasSolar,tariffRulesCount,dmoPrice,annualUsage,annualCLUsage
planId,,,,,,,,,,,,,,,,,,,,,
AMP522157MRE15@EME,ampol,Ampol Energy,ELECTRICITY,MARKET,RESIDENTIAL,Ampol Energy Powering On TOU,2023-12-21T00:00:00.000Z,Energex,TIME_OF_USE,False,...,TIME_OF_USE,False,True,False,False,True,1.0,1969.0,4613.0,NaN
AMP522155MRE15@EME,ampol,Ampol Energy,ELECTRICITY,MARKET,RESIDENTIAL,Ampol Energy Powering On,2023-12-21T00:00:00.000Z,Energex,SINGLE_RATE,False,...,SINGLE_RATE,False,True,False,False,True,1.0,1969.0,4613.0,NaN
AMP522154MRE15@EME,ampol,Ampol Energy,ELECTRICITY,MARKET,RESIDENTIAL,Ampol Energy Powering On with CL,2023-12-21T00:00:00.000Z,Energex,SINGLE_RATE_CONT_LOAD,True,...,SINGLE_RATE_CONT_LOAD,False,True,False,False,True,1.0,2363.0,4412.0,1905.0
AMP522156MRE15@EME,ampol,Ampol Energy,ELECTRICITY,MARKET,RESIDENTIAL,Ampol Energy Powering On TOU with CL,2023-12-21T00:00:00.000Z,Energex,TIME_OF_USE_CONT_LOAD,True,...,TIME_OF_USE_CONT_LOAD,False,True,False,False,True,1.0,2363.0,4412.0,1905.0
OVO704438MRE4@EME,ovo-energy,OVO Energy,ELECTRICITY,MARKET,RESIDENTIAL,The EV Plan Telsa Giveaway,2024-02-09T00:00:00.000Z,SA Power Networks,TIME_OF_USE,False,...,TIME_OF_USE,False,True,True,True,True,1.0,2279.0,4011.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FXP651408MRE1@EME,future-x,Future X Power,ELECTRICITY,MARKET,RESIDENTIAL,Market Offer - Single Rate + CL,2023-08-09T00:00:00.000Z,Essential Energy,SINGLE_RATE_CONT_LOAD,True,...,SINGLE_RATE_CONT_LOAD,False,False,False,False,True,1.0,2977.0,4613.0,2005.0
FXP651397MRE1@EME,future-x,Future X Power,ELECTRICITY,MARKET,RESIDENTIAL,Market Offer - Single Rate,2023-08-09T00:00:00.000Z,Essential Energy,SINGLE_RATE,False,...,SINGLE_RATE,False,False,False,False,True,1.0,2527.0,4613.0,NaN
FXP651402SRE1@EME,future-x,Future X Power,ELECTRICITY,STANDING,RESIDENTIAL,Standing Offer - Single Rate + CL,2023-08-09T00:00:00.000Z,Energex,SINGLE_RATE_CONT_LOAD,True,...,SINGLE_RATE_CONT_LOAD,False,False,False,False,True,1.0,2363.0,4412.0,1905.0


In [ ]:
relevant_plans = relevant_plans[
    (relevant_plans.pricingModel == "SINGLE_RATE") &
    (pd.isna(relevant_plans.tariffPeriod_1_startDate)) &
    (pd.isna(relevant_plans.tariffPeriod_0_demandCharges_0_amount)) &
    (pd.isna(relevant_plans.tariffPeriod_0_singleRate_rates_1_unitPrice))
]

def calculate_demand_charges(planId):
    annualUsage = prices.at[planId, "annualUsage"]
    tariffPrice = relevant_plans.at[planId, "tariffPeriod_0_singleRate_rates_0_unitPrice"] / 100
    return annualUsage * tariffPrice
    
relevant_plans.columns
prices = relevant_plans.iloc[:, :6]
prices["dmoPrice"] = prices.distributor.map(lambda x: standingOffers[(standingOffers.distributionZone == x) & (standingOffers.controlledLoad == False)].iloc[0, 2])
prices["annualUsage"] = prices.distributor.map(lambda x: standingOffers[(standingOffers.distributionZone == x) & (standingOffers.controlledLoad == False)].iloc[0, 3])
prices["supplyCharge"] = relevant_plans.tariffPeriod_0_dailySupplyCharges * 366 / 100
prices["demandCharge"] = prices.index.map(calculate_demand_charges)
prices["estimatedCharge"] = prices.supplyCharge + prices.demandCharge
prices["differenceDmoPrice"] = prices.estimatedCharge - prices.dmoPrice
prices.to_csv("./data/priceEstimates.csv")
prices